In [26]:
import torch

import expt

In [19]:
runs_dir = "/dfs/scratch1/ranjanr/runs/cs336/2024-04-28"
df = expt.scan(runs_dir)

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
df

runs_dir dev lm_size  \
1714360289810038297  /dfs/scratch1/ranjanr/runs/cs336/2024-04-28   0   small   
1714360289859295002  /dfs/scratch1/ranjanr/runs/cs336/2024-04-28   0   small   
1714360460658588621  /dfs/scratch1/ranjanr/runs/cs336/2024-04-28   0   small   

                    vocab_size context_length d_model  d_ff num_layers  \
1714360289810038297      10000            128    None  None       None   
1714360289859295002      10000            128    None  None       None   
1714360460658588621      10000            128    None  None       None   

                    num_heads attn_pdrop residual_pdrop batch_size  \
1714360289810038297      None        0.0            0.0         16   
1714360289859295002      None        0.0            0.0         16   
1714360460658588621      None        0.0            0.0         16   

                    num_train_steps backend device ddp rank local_rank  \
1714360289810038297              10    nccl   cuda   1    0          0   
1714360289859295002              10    nccl   cuda   1    1          1   
1714360460658588621              10    nccl   cuda   0    0          0   

                    world_size  done  
1714360289810038297          2  True  
1714360289859295002          2  True  
1714360460658588621          1  True

In [14]:
base_ts = df.query("ddp == 0").iloc[-1].name

In [15]:
ddp_0_ts = df.query("ddp == 1 and rank == 0").iloc[-1].name
ddp_1_ts = df.query("ddp == 1 and rank == 1").iloc[-1].name

In [20]:
base_store = expt.PathDict(f"{runs_dir}/{base_ts}")
ddp_0_store = expt.PathDict(f"{runs_dir}/{ddp_0_ts}")
ddp_1_store = expt.PathDict(f"{runs_dir}/{ddp_1_ts}")

In [30]:
base_p = torch.cat([p.view(-1) for p in base_store["net"].values()])

In [35]:
ddp_0_p = torch.cat([p.view(-1) for p in ddp_0_store["net"].values()])
ddp_1_p = torch.cat([p.view(-1) for p in ddp_1_store["net"].values()])

In [36]:
torch.allclose(ddp_1_p, ddp_0_p)

True

In [37]:
base_p

tensor([-1.1244, -1.1509, -0.2519,  ...,  0.8883,  1.5857, -1.6798])

In [38]:
ddp_0_p

tensor([-1.1257, -1.1522, -0.2506,  ...,  0.8883,  1.5857, -1.6798])

In [39]:
ddp_0_p == ddp_1_p

tensor([True, True, True,  ..., True, True, True])